In [106]:
import yaml
import jinja2
import nacl.public
from base64 import b64encode
from nacl.public import PrivateKey
from ipaddress import  IPv4Network , IPv6Network ,IPv4Address, IPv6Address
from jinja2 import Template, DebugUndefined

psk_db = {}
keypair_db = {}
port_allocate_db = {}
port_allocate_db_i = {}

def vars_load(var):
    print("INFO: loading old state")
    global psk_db
    global keypair_db
    global port_allocate_db
    global port_allocate_db_i
    if "psk_db" in var:
        psk_db = var["psk_db"]
    if "keypair_db" in var:
        keypair_db = var["keypair_db"]
    if "port_allocate_db" in var:
        port_allocate_db = var["port_allocate_db"]
    if "port_allocate_db_i" in var:
        port_allocate_db_i = var["port_allocate_db_i"]
        
def vars_dump():
    return {
        "psk_db": psk_db,
        "keypair_db": keypair_db,
        "port_allocate_db": port_allocate_db,
        "port_allocate_db_i": port_allocate_db_i,
    }

def keygen():
    private = PrivateKey.generate()
    return (
        b64encode(bytes(private)).decode("ascii"),
        b64encode(bytes(private.public_key)).decode("ascii"),
    )
def get_psk(id,id2):
    dbkey = (min(id,id2),max(id,id2))
    if dbkey in psk_db:
        return psk_db[dbkey]
    else:
        psk ,_ = keygen()
        psk_db[dbkey] = psk
        return psk_db[dbkey]
    
def get_keypair(id):
    if id in keypair_db:
        return keypair_db[id]
    keypair_db[id] = keygen()
    return keypair_db[id]

def allocate_port(id,id2,port_db,port_base):
    if id not in port_db:
        port_db[id] = { id2: port_base }
        return port_db[id][id2]
    elif id2 not in port_db[id]:
        for p in range(port_base, 65535):
            pfound = False
            for _,usedp in port_db[id].items():
                if p == usedp:
                    pfound = True
                    break
            if pfound == False:
                port_db[id][id2] = p 
                return port_db[id][id2]
    return port_db[id][id2]

def get_wg(server,client):
    conftemplate = Template(open('wg/wg.conf').read(), undefined=DebugUndefined)
    setuptemplate = Template(open('wg/wg.sh').read(), undefined=DebugUndefined)
    def renderconf(server,client):
        spri,spub = get_keypair(server["id"])
        cpri,cpub = get_keypair(client["id"])
        if "port" not in server:
            server["port"] = allocate_port(server["name"],client["ifname"],port_allocate_db,server["port_base"]) if server["endpoint"] != "NAT" else 0
        if "port" not in client:
            client["port"] = allocate_port(client["name"],server["ifname"],port_allocate_db,client["port_base"]) if client["endpoint"] != "NAT" else 0
        render_params = {
            'wg': {
                'pri': spri, 
                'port': server["port"],
                "pub": cpub,
                "psk": get_psk(server["id"],client["id"]),
                "endpoint": client["endpoint"] + ":" + str(client["port"]) if client["endpoint"] != "NAT" else None
            },
        }
        return conftemplate.render(**render_params)
    def rendersetup(param):
        render_params = {
            "ifname" : param["ifname"]
        }
        return setuptemplate.render(**render_params)
    conf_s = {
        "up": rendersetup(client),
        "down": "ip link del " + client["ifname"],
        "confs": {".conf": renderconf(server,client) }
    }
    conf_c = {
        "up": rendersetup(server),
        "down": "ip link del " + server["ifname"],
        "confs": {".conf": renderconf(client,server) }
    }
    return conf_s , conf_c

def get_wg_udp2raw(server,client):
    #print(server,client)
    #return
    server["port"]    = allocate_port(server["name"],client["ifname"],port_allocate_db  ,server["port_base"])
    server["port_wg_local"] = allocate_port(server["name"],client["ifname"],port_allocate_db_i,server["port_base_i"])
    client["port_udp2raw_local"] = allocate_port(client["name"],server["ifname"],port_allocate_db_i,client["port_base_i"])
    server_w = {**server}
    client_w = {**client}
    server_w["endpoint"] = "127.0.0.1"
    server_w["port"] = server["port_wg_local"]
    client_w["endpoint"] = "NAT"
    wg_s, _ = get_wg(server_w,client_w)
    server_w["port"] = client["port_udp2raw_local"]
    _ , wg_c = get_wg(server_w,client_w)
    conf_s, conf_c = wg_s, wg_c
    conf_s["up"] += "\n" + f'udp2raw -s -l 0.0.0.0:{server["port"]} -r 127.0.0.1:{server["port_wg_local"]}  -a -k "{get_psk(server["id"],client["id"])[:10]}" --raw-mode faketcp &'
    conf_s["up"] += '\necho $! > {{ confpath }}.pid'
    conf_c["up"] += "\n" + f'udp2raw -c -r {server["endpoint"]}:{server["port"]} -l 127.0.0.1:{client["port_udp2raw_local"]}  -k "{get_psk(server["id"],client["id"])[:10]}" --raw-mode faketcp -a &'
    conf_c["up"] += '\necho $! > {{ confpath }}.pid'
    conf_s["down"] += '\nkill $(cat {{ confpath }}.pid)'
    conf_c["down"] += '\nkill $(cat {{ confpath }}.pid)'
    return conf_s,conf_c

def get_gre(server,client):
    conf_s = {
        "up": f'ip tunnel add {client["ifname"]} mode gre remote { client["endpoint"] } ttl 255',
        "down": "ip link del " + client["ifname"],
        "confs": {}
    }
    conf_c = {
        "up": f'ip tunnel add {server["ifname"]} mode gre remote { server["endpoint"] } ttl 255',
        "down": "ip link del " + server["ifname"],
        "confs": {}
    }
    if server["endpoint"] == "NAT":
        raise ValueError(f'Endpoint can\'t be NAT at gre tunnel: { client["id"] } ->  { server["id"] }' )
    if client["endpoint"] == "NAT":
        raise ValueError(f'Endpoint can\'t be NAT at gre tunnel: { server["id"] } ->  { client["id"] }' )
    return conf_s , conf_c

def get_openvpn(server,client):
    server["port"] = allocate_port(server["name"],client["ifname"],port_allocate_db,server["port_base"])
    ovpncfg = get_openvpn_config(server["id"],client["id"])
    conf_s = {
        "up": 'nohup openvpn --config {{ confpath }}.ovpn >/dev/null >/dev/null 2>&1 &',
        "down": 'kill $(cat {{ confpath }}.pid)',
        "confs": { ".ovpn" : Template(open('ovpn/server.ovpn').read(), undefined=DebugUndefined).render(server=server,client=client),
                  "ca.crt": ovpncfg["ca.crt"],
                  ".crt": ovpncfg["server.crt"],
                  ".key": ovpncfg["server.key"],
                  ".pem": ovpncfg["df.pem"]
                 }
    }
    conf_c = {
        "up": 'nohup openvpn --config {{ confpath }}.ovpn >/dev/null >/dev/null 2>&1 &',
        "down": 'kill $(cat {{ confpath }}.pid)',
        "confs": { ".ovpn" : Template(open('ovpn/client.ovpn').read(), undefined=DebugUndefined).render(server=server,client=client),
                  "ca.crt": ovpncfg["ca.crt"],
                  ".crt": ovpncfg["client.crt"],
                  ".key": ovpncfg["client.key"],
                 }
    }
    return conf_s , conf_c
    
def get_openvpn_config(id,id2):
    return {
        "ca.crt" : "aa",
        "server.crt": "bb",
        "server.key": "cc",
        "client.crt": "dd",
        "client.key": "ee",
        "df.pem": ""
    }
    

def get_v4(id,net):
    first = net[0]
    ip = first + id
    if ip in net:
        return ip
    else:
        raise ValueError(f'{ip} is not in {net}')
def get_v6(id,net):
    first = net[0]
    ip = first + id * (2**64)
    if ip in net:
        return ip
    else:
        raise ValueError(f'{ip} is not in {net}')
def get_v6ll(id,ip):
    return ip + id 

tunnels = {
    None: None,
    "gre": get_gre,
    "wg_udp2raw":get_wg_udp2raw,
    "wg":get_wg,
    "openvpn": get_openvpn,
    
}

tunnelist = list(tunnels.keys())

In [109]:
import jinja2
import yaml
import os
import shutil


import ruamel.yaml
from ruamel.yaml.constructor import SafeConstructor

class PrettySafeLoader(yaml.SafeLoader):
    def construct_python_tuple(self, node):
        return tuple(self.construct_sequence(node))

PrettySafeLoader.add_constructor(
    u'tag:yaml.org,2002:python/tuple',
    PrettySafeLoader.construct_python_tuple)
yaml.Dumper.ignore_aliases = lambda *args : True

if not os.path.isfile("input/all_node.yaml"):
    print("WARN: input/all_node.yaml not found, using default template")
    shutil.copyfile("all_node.yaml", "input/all_node.yaml", follow_symlinks=True)
    
if not os.path.isfile("input/generate_config_func.py"):
    print("WARN: input/generate_config_func.py not found, using default template")
    shutil.copyfile("generate_config_func.py", "input/generate_config_func.py", follow_symlinks=True)

    
gen_conf = ruamel.yaml.safe_load(open("input/all_node.yaml").read())

if os.path.isfile("input/state.yaml"):
    vars_load(yaml.load(open("input/state.yaml").read(), Loader=PrettySafeLoader))

for k,v in gen_conf["node_list"].items():
    gen_conf["node_list"][k]["param"] = {}
    for dk, dv in gen_conf["defaults"].items():
        if dk not in gen_conf["node_list"][k]:
            gen_conf["node_list"][k][dk] = dv
        elif type(dv) == dict:
            gen_conf["node_list"][k][dk] = {**dv, **gen_conf["node_list"][k][dk]}

net4 = IPv4Network(gen_conf["network"]["v4"])
net6 = IPv6Network(gen_conf["network"]["v6"])
net6ll = IPv6Address(gen_conf["network"]["v6ll"])

result = {gen_conf["node_list"][n]["name"]:{"igp_tunnels":{},"bird/ibgp.conf":"","ups":set(),"downs":set()} for n in gen_conf["node_list"]}

def get_iface_full(name,af):
    n = gen_conf["iface_prefix"] + name + af
    if len(n) >= 16:
        raise ValueError(f"The interface name: {n} must be less than 16 (IFNAMSIZ) bytes.")
    return n

def get_tun(node, id2):
    if id2 not in node["tunnel"]:
        return node["tunnel"][-1] , 1
    return node["tunnel"][id2] , 0

INFO: loading old state


In [110]:
server = {'name': 'nj', 'port_base': 42150, 'endpoints': {'-4': '4.cnnj.kskb.eu.org'}, 'tunnel': {-1: 'wg_udp2raw', 10: None}, 'param': {}, 'port_base_i': 36000, 'server_perf': 100, 'MTU': 1360, 'id': 12, 'ifname': 'ibgp-nj-4', 'endpoint': '4.cnnj.kskb.eu.org', 'params': None}
client = {'name': 'tw', 'server_perf': 50, 'endpoints': {'-4': '4.tw.kskb.eu.org', '-6': '6.tw.kskb.eu.org'}, 'tunnel': {-1: 'wg', 8: 'openvpn'}, 'param': {}, 'port_base': 18000, 'port_base_i': 36000, 'MTU': 1360, 'id': 1, 'ifname': 'ibgp-tw-4', 'endpoint': '4.tw.kskb.eu.org', 'params': None}

In [111]:
server

{'name': 'nj',
 'port_base': 42150,
 'endpoints': {'-4': '4.cnnj.kskb.eu.org'},
 'tunnel': {-1: 'wg_udp2raw', 10: None},
 'param': {},
 'port_base_i': 36000,
 'server_perf': 100,
 'MTU': 1360,
 'id': 12,
 'ifname': 'ibgp-nj-4',
 'endpoint': '4.cnnj.kskb.eu.org',
 'params': None}

In [112]:
client

{'name': 'tw',
 'server_perf': 50,
 'endpoints': {'-4': '4.tw.kskb.eu.org', '-6': '6.tw.kskb.eu.org'},
 'tunnel': {-1: 'wg', 8: 'openvpn'},
 'param': {},
 'port_base': 18000,
 'port_base_i': 36000,
 'MTU': 1360,
 'id': 1,
 'ifname': 'ibgp-tw-4',
 'endpoint': '4.tw.kskb.eu.org',
 'params': None}

In [113]:
print(yaml.dump(get_wg_udp2raw(server,client),default_style="|"))

!!python/tuple
- "confs":
    ".conf": |+
      [Interface]
      PrivateKey = 7ZbBahnRMrmH4gJeIyDq0xv7rRN8tL0g9w4VK3o2o2c=
      ListenPort = 36000

      [Peer]
      PublicKey = XdTHdD0LfxtBnCXuD4W24LBa/no98Xi6AZAXOJ4A2VI=
      PresharedKey = nO9QV6MPoPyYMelfFm3Pi/qgpdg54HXjCqmDxyI1Ugw=
      AllowedIPs = 0.0.0.0/0, ::/0
      PersistentKeepalive = 25

  "down": |-
    ip link del ibgp-tw-4
    kill $(cat {{ confpath }}.pid)
  "up": |-
    ip link add dev ibgp-tw-4 type wireguard
    wg setconf ibgp-tw-4 {{ confpath }}.conf
    udp2raw -s -l 0.0.0.0:42150 -r 127.0.0.1:36000  -a -k "nO9QV6MPoP" --raw-mode faketcp &
    echo $! > {{ confpath }}.pid
- "confs":
    ".conf": |+
      [Interface]
      PrivateKey = lUYCgqldb7H1eT4Uki3wylzz5ApklBnKFhyVHg5993c=
      ListenPort = 0

      [Peer]
      PublicKey = ghJ5xDo5zU4EFTPZaqGo/2pWGEXTKG2fat5r3xneNhw=
      PresharedKey = nO9QV6MPoPyYMelfFm3Pi/qgpdg54HXjCqmDxyI1Ugw=
      AllowedIPs = 0.0.0.0/0, ::/0
      Endpoint = 127.0.0.1:36000


In [85]:
port_allocate_db_i

{'nj': {'ibgp-tw-4': 36000}, 'tw': {'ibgp-nj-4': 36000}}

In [29]:
gen_conf

{'output_dir': 'output',
 'defaults': {'port_base': 18000,
  'port_base_i': 36000,
  'server_perf': 100,
  'MTU': 1360,
  'tunnel': {-1: 'wg'}},
 'iface_prefix': 'ibgp-',
 'network': {'v4': '172.22.77.32/28',
  'v6': 'fd28:cb8f:4c92::/48',
  'v6ll': 'fd80::1817:0'},
 'node_list': {1: {'name': 'tw',
   'server_perf': 50,
   'endpoints': {'-4': '4.tw.kskb.eu.org', '-6': '6.tw.kskb.eu.org'},
   'tunnel': {-1: 'wg', 8: 'openvpn'},
   'param': {},
   'port_base': 18000,
   'port_base_i': 36000,
   'MTU': 1360},
  4: {'name': 'jptyo',
   'endpoints': {'-4': '4.jp.kskb.eu.org', '-6': '6.jp.kskb.eu.org'},
   'tunnel': {-1: 'wg', 8: 'gre'},
   'param': {},
   'port_base': 18000,
   'port_base_i': 36000,
   'server_perf': 100,
   'MTU': 1360},
  8: {'name': 'usfmt',
   'endpoints': {'-4': '4.us.kskb.eu.org', '-6': '6.us.kskb.eu.org'},
   'tunnel': {-1: 'wg', 1: 'openvpn', 4: 'gre'},
   'param': {},
   'port_base': 18000,
   'port_base_i': 36000,
   'server_perf': 100,
   'MTU': 1360},
  9: {'nam

In [74]:

for id, node in gen_conf["node_list"].items():
    os.makedirs(gen_conf["output_dir"] + "/" + node["name"], exist_ok=True)
    os.makedirs(gen_conf["output_dir"] + "/" + node["name"] + "/igp_tunnels", exist_ok=True)
    os.makedirs(gen_conf["output_dir"] + "/" + node["name"] + "/bird", exist_ok=True)

for id, node in gen_conf["node_list"].items():
    for id2, node2 in gen_conf["node_list"].items():
        if id == id2:
            continue
        ibgptemplate = jinja2.Template(open('bird_ibgp.conf').read())
        result[node["name"]]["bird/ibgp.conf"] += ibgptemplate.render(name = gen_conf["iface_prefix"] + node2["name"],ip=get_v6(id2,net6))

        for af, end in node["endpoints"].items():
            if af not in node2["endpoints"]: # process only if both side has same af
                continue
            if node["endpoints"][af] == "NAT" and node2["endpoints"][af] == "NAT": # skip if both side are NATed
                continue
            tuntype1, wildcard1 = get_tun(node,id2)
            tuntype2, wildcard2 = get_tun(node2,id)
            if (wildcard1,tunnelist.index(tuntype1)) > (wildcard2,tunnelist.index(tuntype2)):
                tuntype = tuntype2
            else:
                tuntype = tuntype1
            if tuntype1 != tuntype2:
                print("WARN: Tunnel type not match: {s}->{e}:{t1} , {e}->{s}:{t2}, selecting {tun}".format(s=id,e=id2,t1=tuntype1,t2=tuntype2,tun=tuntype))
            if tuntype == None:
                continue
            side_a = {
                **node,
                "id": id,
                "ifname": get_iface_full(node["name"],af),
                "endpoint": node["endpoints"][af],
                "params": node["param"][tuntype] if tuntype in node["param"] else None
            }
            side_b = {
                **node2,
                "id": id2,
                "ifname": get_iface_full(node2["name"],af),
                "endpoint": node2["endpoints"][af],
                "params": node2["param"][tuntype] if tuntype in node2["param"] else None
            }
            
            setiptemplate = jinja2.Template(open('setip.sh').read())
            if side_a["endpoint"] == "NAT":
                continue
            try:
                if side_b["endpoint"] == "NAT" or (node["server_perf"],id) >= (node2["server_perf"],id2):
                    aconf, bconf = tunnels[tuntype](side_a,side_b)
                else:
                    bconf, aconf = tunnels[tuntype](side_b,side_a)
                def postprocess(conf,side,idd,nod,side2):
                    conf["up"] += "\n" + setiptemplate.render(ifname=side2["ifname"],MTU=nod["MTU"],ipv4=get_v4(idd,net4),ipv6=get_v6(idd,net6),ipv6ll=get_v6ll(idd,net6ll))
                    conf["up"] = jinja2.Template(conf["up"]).render(confpath = "igp_tunnels/" + side2["ifname"])
                    conf["down"] = jinja2.Template(conf["down"]).render(confpath = "igp_tunnels/" + side2["ifname"])
                    for ck,cv in conf["confs"].items():
                        conf["confs"][ck] = jinja2.Template(cv).render(confpath = "igp_tunnels/" + side2["ifname"])
                postprocess(aconf,side_a,id,node,side_b)
                postprocess(bconf,side_b,id2,node2,side_a)
            except ValueError as e:
                print("WARN: " + str(e))
                continue
            if "confs" in aconf:
                result[gen_conf["node_list"][id ]["name"]]["igp_tunnels"][side_b["ifname"]] = aconf["confs"]
            if "confs" in bconf:
                result[gen_conf["node_list"][id2]["name"]]["igp_tunnels"][side_a["ifname"]] = bconf["confs"]
            result[gen_conf["node_list"][id ]["name"]]["ups"].add(aconf["up"])
            result[gen_conf["node_list"][id2]["name"]]["ups"].add(bconf["up"])
            result[gen_conf["node_list"][id ]["name"]]["downs"].add(aconf["down"])
            result[gen_conf["node_list"][id2]["name"]]["downs"].add(bconf["down"])

        
for s,sps in result.items():
    for e , confs in sps["igp_tunnels"].items():
        for ext, content in confs.items():
            open(gen_conf["output_dir"] + "/" + s + "/igp_tunnels/" + e + ext , "w").write(content)
    render_params = {
        'allow_ip': {
            'v4': gen_conf["network"]["v4"], 
            'v6': gen_conf["network"]["v6"], 
        },
        'interfaces': list(sps["igp_tunnels"].keys())
    }
    babeldtemplate = jinja2.Template(open('babeld.conf').read())
    babeldconf = babeldtemplate.render(**render_params)
    open(gen_conf["output_dir"] + "/" + s + "/babeld.conf" , "w").write(babeldconf)
    open(gen_conf["output_dir"] + "/" + s + "/up.sh" , "w").write( jinja2.Template(open('up.sh').read()).render(ups = sps["ups"]))
    open(gen_conf["output_dir"] + "/" + s + "/down.sh" , "w").write( jinja2.Template(open('down.sh').read()).render(downs = sps["downs"]))
    open(gen_conf["output_dir"] + "/" + s + "/bird/ibgp.conf" , "w").write(sps["bird/ibgp.conf"])
    
open("input/state.yaml","w").write(ruamel.yaml.dump(vars_dump()))

WARN: Tunnel type not match: 1->12:wg , 12->1:wg_udp2raw, selecting wg_udp2raw
{'name': 'nj', 'port_base': 42150, 'endpoints': {'-4': '4.cnnj.kskb.eu.org'}, 'tunnel': {-1: 'wg_udp2raw', 10: None}, 'param': {}, 'port_base_i': 36000, 'server_perf': 100, 'MTU': 1360, 'id': 12, 'ifname': 'ibgp-nj-4', 'endpoint': '4.cnnj.kskb.eu.org', 'params': None} {'name': 'tw', 'server_perf': 50, 'endpoints': {'-4': '4.tw.kskb.eu.org', '-6': '6.tw.kskb.eu.org'}, 'tunnel': {-1: 'wg', 8: 'openvpn'}, 'param': {}, 'port_base': 18000, 'port_base_i': 36000, 'MTU': 1360, 'id': 1, 'ifname': 'ibgp-tw-4', 'endpoint': '4.tw.kskb.eu.org', 'params': None}


TypeError: cannot unpack non-iterable NoneType object